# 1. Simulating with Prognostics Models

One of the most basic of functions for models is simulation. Simulation is the process of predicting the evolution of [system's state](https://nasa.github.io/progpy/glossary.html#term-state) with time. Simulation is the foundation of prediction (see 9. Prediction). Unlike full prediction, simulation does not include uncertainty in the state and other product (e.g., [output](https://nasa.github.io/progpy/glossary.html#term-output)) representation.

The first section introduces simulating to a specific time (e.g., 3 seconds), using the `simulate_to` method. The second section introduces the concept of simulating until a threshold is met rather than a defined time, using `simulate_to_threshold`. The third section makes simulation more concrete with the introduction of [future loading](https://nasa.github.io/progpy/glossary.html#term-future-load). The sections following these introduce various advanced features that can be used in simulation.

Note: Before running this example make sure you have ProgPy installed and up to date.

## Basic Simulation to a Time

Let's go through a basic example simulating a model to a specific point in time. In this case we are using the ThrownObject model. ThrownObject is a basic model of an object being thrown up into the air (with resistance) and returning to the ground.

First we import the model from ProgPy's models subpackage (see 3. Included Models) and create a model instance.

In [ ]:
from progpy.models import ThrownObject
m = ThrownObject()

Now we simulate this model for three seconds. To do this we use the [`simulate_to`](https://nasa.github.io/progpy/api_ref/prog_models/PrognosticModel.html#prog_models.PrognosticsModel.simulate_to) method.

In [ ]:
results = m.simulate_to(3)

It's that simple! We've simulated the model forward three seconds. Let's look in a little more detail at the returned results. 

Simulation results consists of 5 different types of information, described below:
* **times**: the time corresponding to each value.
* **[inputs](https://nasa.github.io/progpy/glossary.html#term-input)**: Control or loading applied to the system being modeled (e.g., current drawn from a battery). Input is frequently denoted by u.
* **[states](https://nasa.github.io/progpy/glossary.html#term-state)**: Internal variables (typically hidden states) used to represent the state of the system. Can be same as inputs or outputs but do not have to be. State is frequently denoted as `x`.
* **[outputs](https://nasa.github.io/progpy/glossary.html#term-output)**: Measured sensor values from a system (e.g., voltage and temperature of a battery). Can be estimated from the system state. Output is frequently denoted by `z`.
* **[event_states](https://nasa.github.io/progpy/glossary.html#term-event-state)**: Progress towards [event](https://nasa.github.io/progpy/glossary.html#term-event) occurring. Defined as a number where an event state of 0 indicates the event has occurred and 1 indicates no progress towards the event (i.e., fully healthy operation for a failure event). For a gradually occurring event (e.g., discharge) the number will progress from 1 to 0 as the event nears. In prognostics, event state is frequently called “State of Health”.

In this case, times are the start and beginning of the simulation ([0, 3]), since we have not yet told the simulator to save intermediate times. The ThrownObject model doesn't have any way of controlling or loading the object, so there are no inputs. The states are position (`x`) and velocity (`v`). This model assumes that you can measure position, so the outputs are just position (`x`). The two events for this model are `falling` (i.e., if the object is falling towards the earth) and `impact` (i.e., the object has impacted the ground). For a real prognostic model, events might be failure modes or warning thresholds.

Now let's inspect the results. First, let's plot the outputs (position)

In [ ]:
fig = results.outputs.plot()

This is a model of an object thrown in the air, so we generally expect its path to follow a parabola, but what we see above is linear. This is because there are only two points, the start (0s) and the end (3s). To see the parabola we need more points. This is where `save_freq` and `save_pts` come into play. 

`save_freq` is an argument in simulation that specifies a frequency at which you would like to save the results (e.g., 1 seconds), while `save_pts` is used to specify specific times that you would like to save the results (e.g., [1.5, 2.5, 3, 5] seconds).

Now let's repeat the simulation above with a save frequency and plot the results.

In [ ]:
results = m.simulate_to(3, save_freq=0.5)
fig = results.outputs.plot(ylabel='Position (m)')

Now we can see the start of the parabola path we expected. We dont see the full parabola because we stopped simulation at 3 seconds.

If you look at results.times, you can see that the results were saved every 0.5 seconds during simulation

In [ ]:
print(results.times)

Next, let's look at the event_states (i.e., `falling` and `impact`).

In [ ]:
fig = results.event_states.plot()

Here we see that the `falling` event state decreased linerally with time, and was approaching 0. This shows that it was nearly falling when we stopped simulation. The `impact` event state remained at 1, indicating that we had not made any progress towards impact. With this model, `impact` event state only starts decreasing as the object falls. 

Finally, let's take a look at model states. In this case the two states are position (`x`) and velocity (`v`).

In [ ]:
fig = results.states.plot()

Position will match the output exactly and velocity (`v`) decreases nearly linerally with time due to the constant pull of gravity. The slight non-linerality is due to the effects of drag.

This is a basic example of simulating to a set time. This is useful information for inspecting or analyzing the behavior of a model or the degredation of a system. There are many useful features that allow for complex simulation, described in the upcoming sections. 

Note that this is an example problem. In most cases, the system will have inputs, in which case simulation will require future loading (see Future Loading section, below), and simulation will not be until a time, but until a threshold is met. Simulating to a threshold will be described in the next section.

## Simulating to Threshold

In the first section we introduced simulating to a set time. For most applications, users are not interested in the system evolution over a certain time period, but instead in simulating to some event of interest.

In this section we will introduce the concept of simulating until an event occurs. This section builds upon the concepts introduced in the previous section.

Just like in the previous section, we will start by preparing the ThrownObject model. 

In [ ]:
from progpy.models import ThrownObject
m = ThrownObject()

If you recall, the ThrownObject model is of an object thrown into the air. The model has two events, `impact` and `falling`. In real prognostic models, these events will likely correspond with some failure, fault, or warning threshold. That said, events can be any event of interest that a user would like to predict. 

Now let's repeat the simulation from the previous example, this time simulating until an event has occured by using the [`simulate_to_threshold`](https://nasa.github.io/progpy/api_ref/prog_models/PrognosticModel.html#prog_models.PrognosticsModel.simulate_to_threshold) method.

In [ ]:
results = m.simulate_to_threshold(save_freq=0.5)
fig = results.outputs.plot(ylabel='Position (m)')
fig = results.event_states.plot()

Note that simulation continued beyond the 3 seconds used in the first section. Instead simulation stopped at 4 seconds, at which point the `falling` event state reached 0 and the position (`x`) reached the apogee of its path.

By default, `simulate_to_threshold` simulates until the first event occurs. In this case, that's `falling` (i.e., when the object begins falling). For this model `falling` will always occur before `impact`, but for many models you won't have such a strict ordering of events. 

For users interested in when a specific event is reached, you can indicate which event(s) you'd like to simulate to using the `threshold_keys` argument. For example,

In [ ]:
results = m.simulate_to_threshold(save_freq=0.5, threshold_keys='impact')
fig = results.outputs.plot(ylabel='Position (m)')
fig = results.event_states.plot()

Now the model simulated past the `falling` event untill the `impact` event occured. `threshold_keys` accepts a single event, or a list of events, so for models with many events you can specify a list of events where any will stop simulation.

Frequently users are interested in simulating to a threshold, only if it occurs within some horizon of interest, like a mission time or planning horizon. This is accomplished with the `horizon` keyword argument. 

For example, if we were only interested in events occuring in the next 7 seconds we could set `horizon` to 7, like below:

In [ ]:
results = m.simulate_to_threshold(save_freq=0.5, threshold_keys='impact', horizon=7)
fig = results.outputs.plot(ylabel='Position (m)')
fig = results.event_states.plot()

Notice that now simulation stopped at 7 seconds, even though the event had not yet occured. If we use a horizon after the event, like 10 seconds, then simulation stops at the event.

In [ ]:
results = m.simulate_to_threshold(save_freq=0.5, threshold_keys='impact', horizon=10)
fig = results.outputs.plot(ylabel='Position (m)')
fig = results.event_states.plot()

The 7 and 10 second horizon is used as an example. In most cases, the simulation horizon will be much longer. For example, you can imagine a user who's interested in prognostics for a one hour drone flight might set the horizon to a little over an hour. A user who has a month-long maintenance scheduling window might chose a horizon of one month. 

It is good practice to include a horizon with most simulations to prevent simulations continuing indefinitely for the case where the event never happens.

One final note: you can also use the print and progress options to track progress during long simulations, like below:

In [ ]:
results = m.simulate_to_threshold(save_freq=0.5, threshold_keys='impact', print=True, progress=True)

For most users running this in Jupyter notebook, the output will be truncated, but it gives an idea of what would be shown when selecting these options.

This is a basic example of simulating to an event. However, this is still just an example. Most models will have some form of input or loading. Simulating these models is described in the following section. The remainder of the sections go through various features for customizing simulation further.

## Future Loading

## Step Size

## Noise

## Vectorized Simulation

## Configuring Simulation
* t0, x
* integration_method

## Conclusions
...

simulate_to_event(), events